This file gets information on historic quarterly reports for all companies listed in the SP500

In [19]:
import pandas as pd
from time import sleep
from datetime import datetime
import requests
import os
import json

This code loadsthe CIK values from Wikipedia

In [5]:
# URL of the Wikipedia page
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Read the tables on the Wikipedia page
tables = pd.read_html(wiki_url)

# The first table on the page contains the S&P 500 companies
sp500_table = tables[0]

This code gets JSON files with historic 10 Q filings for all items in the list above

In [6]:
# downloading files from SEC
def download_file(url="https://www.sec.gov/include/ticker.txt", filename="CIK.tsv", user_agent="Michelle LiuWatts admin@hercompany.com"):
    headers = {"User-Agent": user_agent}
    response = requests.get(url, headers=headers)
    sleep(0.101)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        return f"File downloaded successfully as {filename}"
    else:
        return f"Failed to download file: {response.status_code}"

In [9]:
new = exist = failed = 0
for i, row in sp500_table.iterrows():
    cik = str(row['CIK']).zfill(10)
    ticker = row['Symbol']
    save_dir = "sec_data"
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    if not os.path.exists(f"{save_dir}/{ticker}.json"):
        result = download_file(url, f"{save_dir}/{ticker}.json")
        new = new + 1
    else:
        result = f"{ticker} already downloaded"
        exist = exist + 1
        # print(result)
    if "Failed" in result:
        print(f"{ticker} failed to download")
        failed = failed + 1
print(f"Downloaded {new} new files, {exist} files already existed, and {
      failed} files failed to download")

Downloaded 0 new files, 503 files already existed, and 0 files failed to download


Now to extract data from the saved Jsons

In [12]:
# Define metrics and their units as in the sec json files
eps_metrics = {
    "EarningsPerShareDiluted": "USD/shares",
    "IncomeLossFromContinuingOperationsPerDilutedShare": "USD/shares",
    "IncomeLossFromDiscontinuedOperationsNetOfTaxPerDilutedShare": "USD/shares",
    "WeightedAverageNumberOfDilutedSharesOutstanding": "shares",
    "NetIncomeLoss": "USD",
    "NetIncomeLossFromContinuingOperationsAvailableToCommonShareholdersDiluted": "USD",
    "NetIncomeLossFromDiscontinuedOperationsAvailableToCommonShareholdersDiluted": "USD"

}

In [13]:

def extract_metrics(metrics_and_units, directory="./sec_data"):
    """
    Extracts specified metrics and their units from JSON files in the current directory.

    Args:
        metrics_and_units: A dictionary where keys are metric names 
                           and values are their corresponding units.

    Returns:
        A pandas DataFrame containing the extracted data.
    """
    data = []
    # current_directory = os.getcwd()

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            ticker = os.path.splitext(filename)[0]

            try:
                with open(file_path, 'r') as file:
                    json_data = json.load(file)
                    us_gaap_elements = json_data.get(
                        'facts', {}).get('us-gaap', {})

                    for metric, unit in metrics_and_units.items():
                        metric_data = us_gaap_elements.get(
                            metric, {}).get('units', {})
                        if unit in metric_data:
                            for entry in metric_data[unit]:
                                entry_data = {
                                    'ticker': ticker, 'metric': metric}
                                entry_data.update(entry)
                                data.append(entry_data)

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {filename}: {e}")

    return pd.DataFrame(data)

In [15]:
# Extract data using the defined metrics
extract_metrics(eps_metrics).to_csv("EPS_fact.csv", index=False)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002289B8E6FF0>>
Traceback (most recent call last):
  File "C:\Users\Eshwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1535, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


In [24]:
df = pd.read_csv("EPS_fact.csv")


def days_between_dates(date1_str, date2_str, date_format="%Y-%m-%d"):
    if pd.isnull(date1_str) or pd.isnull(date2_str):
        return None
    date1 = datetime.strptime(date1_str, date_format)
    date2 = datetime.strptime(date2_str, date_format)
    delta = date2 - date1
    return abs(delta.days)


# Apply the function to the DataFrame
df['period_days'] = df.apply(lambda row: days_between_dates(
    row['start'], row['end']), axis=1)

The idea is to create a line per event to track price changees across all spx stocks due to that event
An event can be:
1. declaration of quarterly reports of a SPX stock
2. interest rate changes 
3. declaration of economic indicator reports 
4. news ?? (unknown now)
how is this useful ?
